# Circle

Consider the following scheme: distribute $n$ variables uniformly randomly around a circle then generate $m$ clauses of $a$ variables each where each clause has a random "center" point on the circle from which all variables in the clause must be within a certain distance $w$.

Observe the time taken to solve instances of different $w$.

In [1]:
# set up
! rm -r experiment; mkdir -p experiment
%cd experiment

/Users/bnran/dev/sat-locality/notebooks/experiment


In [21]:
# parameters
a = 5
m = 100000
n = 100000

## Generate instances

For $w = 0.1, 0.2, 0.3, 0.4, 0.5$:

In [ ]:
%%time
%%bash -s "$a" "$m" "$n"
seq 5 | parallel sl -n "$3" -m "$2" -a "$1" -w "0.{}" -o "instance-{}.cnf" -q

In [ ]:
%ls

In [ ]:
! head instance-1.cnf

## Solve

In [ ]:
%%time
%%bash
for w in $(seq 5); do 
    cat instance-${w}.cnf | docker run --rm -i msoos/cryptominisat:v2 > result-${w}.out
done

In [ ]:
%ls

## Analyze

In [ ]:
%%bash
for w in $(seq 5); do
    printf "0.${w},"
    cat result-${w}.out | grep 'Total time' | sed 's/[^0-9\.]//g' | xargs echo -n
    printf ","
    cat result-${w}.out | grep '^s' | cut -d' ' -f 2
done | tee times.tsv

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('times.tsv', header=None)
W = data[:][0]
T = data[:][1]

fig, ax = plt.subplots(1, 1)

ax.plot(W, T, zorder=10)
ax.grid(True, zorder=5)
plt.show()